In [1]:
import pandas as pd
pd.set_option('display.max_columns', None, 'display.max_rows', None)

In [17]:
# This was run to convert the excel file to csv

# xl_fname ='EAMMi2-Data1.2.xlsx'
# df_raw = pd.read_excel(xl_fname)
# df_raw.columns = map(str.lower, df_raw.columns)
# df_raw.to_csv('EAMMi2-Data1.2.csv', header=True, index=None)

In [2]:
fname = 'EAMMi2-Data1.2.csv'

In [17]:
df_raw = pd.read_csv(fname)

In [5]:
def filter_attention(df):
    '''These columns were used to ensure people were paying attention
    Filter on these conditions, then drop the columns'''
    
    cond1 = df['usdream_3'] == 1
    cond2 = df['attenion2'] == 7
    df = df[cond1 & cond2]
    df.drop(['usdream_3', 'attenion2'], axis=1, inplace=True)
    
    return df.reset_index(drop=True)

In [6]:
def timing(df):
    '''Calculate the duration (in min) for each section,
    then drop the original columns'''
    
    first_click = [c for c in df.columns if 'first click' in c]
    last_click = [c for c in df.columns if 'last click' in c]
    del_cols = [c for c in df.columns if 'click' in c or 'submit' in c] + ['duration (in seconds)']
    
    # Calc total duration in minutes
    df.insert(0, 'duration_min', df['duration (in seconds)'] / 60)
    
    # Calc minutes for each section
    for first, last in zip(first_click, last_click):
        idx = df.columns.get_loc(last)
        new_title = first[:4] + 'duration'
        minutes = (df[last]-df[first]) / 60
        
        df.insert(idx, new_title, minutes)
        df[new_title] = df[new_title].round(2)
    
    # Drop cols/ nan rows
    df = df[pd.notnull(df['q65_last click'])]
    df = df[pd.notnull(df['q81_last click'])]
    df.drop(del_cols, axis=1, inplace=True)
    
    return df.reset_index(drop=True)

In [7]:
def drop(df):
    '''Drop these columns for various reasons'''
    
    # npi questions are not exclusive, but instructions call for binary response 
    # Otherwise lots of encoding if keeping them in
    npis = [c for c in df.columns if 'npi' in c]
    
    del_cols = ['startdate','enddate','status', 'progress','recordeddate',
                'responseid', 'recipientlastname','recipientfirstname','recipientemail',
                'externalreference','distributionchannel','informedconsent','president',
                'transgres','relation','relation_10_text','fault','comments','q14_6',
                'q14_6_text','school','q81','affiliation','school_coded','race_6_text',
                'q78_duration','marriage3']
    total_cols = npis + del_cols
    
    return df.drop(total_cols, axis=1) 

In [8]:
def rename(df):
    
    # Rename various cols for readability
    rename_dict = {
        'adult_q': 'moa_adult',
        'q65_duration': 'moa_duration',
        'q66_duration': 'idea_duration',
        'q74_duration': 'politic_duration',
        'q67_duration': 'swb_duration',
        'q68_duration': 'mindful_duration',
        'belnow': 'belong_now',
        'q72_duration': 'belong_duration',
        'q77_duration': 'efficacy_duration',
        'q96_duration': 'support_duration',
        'q80_duration': 'socmedia_duration',
        'q73_duration': 'usdream_duration',
        'q78_duration': 'transgres_text_duration',
        'q79_duration': 'transgres_duration',
        'q76_duration': 'exploit_duration',
        'q71_duration': 'disability_duration',
        'q70_duration': 'phys_duration',
        'q69_duration': 'stress_duration',
        'q75_duration': 'marriage_duration',
        'q81_duration': 'demo_duration',
        'freq': 'transgres_freq',
        'common': 'transgres_common',
        'age': 'demo_age',
        'q82': 'demo_mil',
        'q83': 'demo_mil_years',
        'place2': 'demo_us',
        'q80': 'demo_us_years',
        'phys_sx_biaschec': 'physsx_biascheck',
        'phys_sym_bias_dummy,': 'physsx_bias_dummy'
    }
    
    # Rename disability cols
    dis = [c for c in df.columns if 'q11' in c or 'q14' in c or 'q10' in c]
    new_names = ['disability' + x[1:] for x in dis]
    dis_dict = {old:new for old,new in zip(dis, new_names)}
    
    total = {**rename_dict, **dis_dict}

    return df.rename(columns=total)

In [9]:
def drop_enc(df):
    '''Until these columns can be encoded, they will be dropped'''
    
    enc_cols = ['politics','party','feel','marriage5','sex','edu','sibling','race','income','place']
    
    # Include corresponding coumns
    plus = ['politic_duration']
    total = enc_cols+plus
    
    return df.drop(total, axis=1)

In [10]:
def remap(df):
    df = df.copy()
    
    disability = [c for c in df.columns if 'disability' in c]   
    df[disability].replace({2:0}, inplace=True)
    df['moa_adult'].replace({3: 1, 1: 3}, inplace=True)
    df['demo_us'].replace({2:0}, inplace=True)
    df['demo_mil'].replace({3: 0}, inplace=True)
    
    return df

In [11]:
def fill_nas(df):
    df = df.copy()
    
    disability = [c for c in df.columns if 'disability' in c]   
    df[disability] = df[disability].fillna(0)
    df['demo_age'] = df['demo_age'].fillna(df['demo_age'].median())
    df['demo_mil'] = df['demo_mil'].fillna(0)
    df['demo_us'] = df['demo_us'].fillna(0)
    df['demo_mil_years'] = df['demo_mil_years'].fillna(0)
    df['demo_us_years'] = df['demo_us_years'].fillna(0)
    df = df.fillna(df.median())
    
    return df

In [12]:
def total_process(df):
    df = filter_attention(df)
    df = timing(df)
    df = drop(df)
    df = rename(df)
    df = drop_enc(df)
    df = remap(df)
    df = fill_nas(df)

    return df

In [23]:
df = total_process(df_raw.copy());

In [20]:
# df.isna().sum()
# df.head()

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2826 entries, 0 to 2825
Columns: 219 entries, duration_min to response_bias_sum
dtypes: float64(197), int64(22)
memory usage: 4.7 MB


In [22]:
# Lists of individual sections

moa = [c for c in df.columns if 'moa' in c]
idea = [c for c in df.columns if 'idea' in c]
swb = [c for c in df.columns if 'swb' in c]
mindful = [c for c in df.columns if 'mindful' in c]
belong = [c for c in df.columns if 'belong' in c]
efficacy = [c for c in df.columns if 'efficacy' in c]
support = [c for c in df.columns if 'support' in c]
socmedia = [c for c in df.columns if 'socmedia' in c]
usdream = [c for c in df.columns if 'usdream' in c]
transgres = [c for c in df.columns if 'transgres' in c]
exploit = [c for c in df.columns if 'exploit' in c]
disability = [c for c in df.columns if 'disability' in c]
phys = [c for c in df.columns if 'phys' in c]
stress = [c for c in df.columns if 'stress' in c]
marriage = [c for c in df.columns if 'marriage' in c]
demo = [c for c in df.columns if 'demo' in c]